In [41]:
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Flatten, Dense, Dropout
from tensorflow.python.keras.applications.resnet50 import ResNet50
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

In [42]:
# 資料路徑
DATASET_PATH  = './data/data/kaggle_dogcat'

# 影像大小
IMAGE_SIZE = (224, 224)

# 影像類別數
NUM_CLASSES = 2

# 若 GPU 記憶體不足，可調降 batch size 或凍結更多層網路
BATCH_SIZE = 8

# 凍結網路層數
FREEZE_LAYERS = 2

# Epoch 數
NUM_EPOCHS = 20

# 模型輸出儲存的檔案
WEIGHTS_FINAL = 'model-resnet50-final.h5'

In [43]:
# 透過 data augmentation 產生訓練與驗證用的影像資料
train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_batches = train_datagen.flow_from_directory(DATASET_PATH + '/train',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=True,
                                                  batch_size=BATCH_SIZE)

valid_datagen = ImageDataGenerator()
valid_batches = valid_datagen.flow_from_directory(DATASET_PATH + '/val',
                                                  target_size=IMAGE_SIZE,
                                                  interpolation='bicubic',
                                                  class_mode='categorical',
                                                  shuffle=False,
                                                  batch_size=BATCH_SIZE)

Found 3200 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [44]:
# 輸出各類別的索引值
for cls, idx in train_batches.class_indices.items():
    print('Class #{} = {}'.format(idx, cls))

Class #0 = cats
Class #1 = dogs


In [45]:
#以訓練好的 ResNet50 為基礎來建立模型，
# 捨棄 ResNet50 頂層的 fully connected layers
net = ResNet50(include_top=False, weights='imagenet', input_tensor=None,
               input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
x = net.output
x = Flatten()(x)

# 增加 DropOut layer
x = Dropout(0.5)(x)

# 增加 Dense layer，以 softmax 產生個類別的機率值
output_layer = Dense(NUM_CLASSES, activation='softmax', name='softmax')(x)

# 設定凍結與要進行訓練的網路層
net_final = Model(inputs=net.input, outputs=output_layer)
for layer in net_final.layers[:FREEZE_LAYERS]:
    layer.trainable = False
for layer in net_final.layers[FREEZE_LAYERS:]:
    layer.trainable = True

# 使用 Adam optimizer，以較低的 learning rate 進行 fine-tuning
net_final.compile(optimizer=Adam(lr=1e-5),
                  loss='categorical_crossentropy', metrics=['accuracy'])


In [46]:
# 輸出整個網路結構
print(net_final.summary())

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
____________________________________________________________________________________________

In [47]:
reduce_lr = ReduceLROnPlateau(factor=0.5, 
                              min_lr=1e-12, 
                              monitor='val_loss', 
                              patience=5, 
                              verbose=1)

In [48]:
# 訓練模型
net_final.fit_generator(train_batches,
                        steps_per_epoch = train_batches.samples // BATCH_SIZE,
                        validation_data = valid_batches,
                        validation_steps = valid_batches.samples // BATCH_SIZE,
                        epochs = NUM_EPOCHS,
                        callbacks=[reduce_lr])


Epoch 1/20
400/400 [==============================] - 57s 143ms/step - loss: 0.8826 - acc: 0.7303 - val_loss: 0.1320 - val_acc: 0.9463
Epoch 2/20
400/400 [==============================] - 39s 99ms/step - loss: 0.3889 - acc: 0.8809 - val_loss: 0.1108 - val_acc: 0.9650
Epoch 3/20
400/400 [==============================] - 40s 99ms/step - loss: 0.3263 - acc: 0.8994 - val_loss: 0.0709 - val_acc: 0.9725
Epoch 4/20
400/400 [==============================] - 40s 99ms/step - loss: 0.2630 - acc: 0.9209 - val_loss: 0.0582 - val_acc: 0.9787
Epoch 5/20
400/400 [==============================] - 39s 99ms/step - loss: 0.2673 - acc: 0.9216 - val_loss: 0.0513 - val_acc: 0.9800
Epoch 6/20
400/400 [==============================] - 40s 99ms/step - loss: 0.2139 - acc: 0.9384 - val_loss: 0.0494 - val_acc: 0.9825
Epoch 7/20
400/400 [==============================] - 40s 99ms/step - loss: 0.1986 - acc: 0.9362 - val_loss: 0.0518 - val_acc: 0.9812
Epoch 8/20
400/400 [==============================] - 40s 99m

In [49]:
# 儲存訓練好的模型
net_final.save(WEIGHTS_FINAL)

In [50]:
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.preprocessing import image
import sys
import numpy as np
import os

In [51]:
# 讀取圖檔路徑
test_dir = "./data/data/kaggle_dogcat/test"
test_list = os.listdir(test_dir)
test_list.sort()
files = []
for f in test_list:
    files.append(test_dir+"/"+f)
    
# 載入訓練好的模型
net = load_model('model-resnet50-final.h5')

cls_list = ['cats', 'dogs']

# 辨識每一張圖
for f in files:
    img = image.load_img(f, target_size=(224, 224))
    if img is None:
        continue
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    pred = net.predict(x)[0]
    top_inds = pred.argsort()[::-1][:5]
    print(f)
    for i in top_inds:
        print('    {:.3f}  {}'.format(pred[i], cls_list[i]))

W0815 06:11:07.215078 140597708195648 deprecation.py:506] From /home/simon/tf_workspace/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0815 06:11:07.216233 140597708195648 deprecation.py:506] From /home/simon/tf_workspace/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling Ones.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0815 06:11:07.278262 140597708195648 deprecation.py:506] From /home/simon/tf_workspace/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.p

./data/data/kaggle_dogcat/test/000.jpg
    1.000  cats
    0.000  dogs
./data/data/kaggle_dogcat/test/001.jpg
    0.999  dogs
    0.001  cats
./data/data/kaggle_dogcat/test/002.jpg
    1.000  cats
    0.000  dogs
./data/data/kaggle_dogcat/test/003.jpg
    1.000  dogs
    0.000  cats
./data/data/kaggle_dogcat/test/004.jpg
    1.000  cats
    0.000  dogs
./data/data/kaggle_dogcat/test/005.jpg
    1.000  cats
    0.000  dogs
./data/data/kaggle_dogcat/test/006.jpg
    1.000  dogs
    0.000  cats
./data/data/kaggle_dogcat/test/007.jpg
    1.000  cats
    0.000  dogs
./data/data/kaggle_dogcat/test/008.jpg
    1.000  cats
    0.000  dogs
./data/data/kaggle_dogcat/test/009.jpg
    1.000  cats
    0.000  dogs
./data/data/kaggle_dogcat/test/010.jpg
    1.000  dogs
    0.000  cats
./data/data/kaggle_dogcat/test/011.jpg
    1.000  cats
    0.000  dogs
./data/data/kaggle_dogcat/test/012.jpg
    1.000  dogs
    0.000  cats
./data/data/kaggle_dogcat/test/013.jpg
    1.000  cats
    0.000  dogs
./data

./data/data/kaggle_dogcat/test/120.jpg
    1.000  cats
    0.000  dogs
./data/data/kaggle_dogcat/test/121.jpg
    1.000  dogs
    0.000  cats
./data/data/kaggle_dogcat/test/122.jpg
    1.000  cats
    0.000  dogs
./data/data/kaggle_dogcat/test/123.jpg
    1.000  cats
    0.000  dogs
./data/data/kaggle_dogcat/test/124.jpg
    1.000  cats
    0.000  dogs
./data/data/kaggle_dogcat/test/125.jpg
    1.000  dogs
    0.000  cats
./data/data/kaggle_dogcat/test/126.jpg
    1.000  dogs
    0.000  cats
./data/data/kaggle_dogcat/test/127.jpg
    1.000  dogs
    0.000  cats
./data/data/kaggle_dogcat/test/128.jpg
    0.887  dogs
    0.113  cats
./data/data/kaggle_dogcat/test/129.jpg
    0.999  cats
    0.001  dogs
./data/data/kaggle_dogcat/test/130.jpg
    1.000  dogs
    0.000  cats
./data/data/kaggle_dogcat/test/131.jpg
    1.000  dogs
    0.000  cats
./data/data/kaggle_dogcat/test/132.jpg
    1.000  cats
    0.000  dogs
./data/data/kaggle_dogcat/test/133.jpg
    1.000  cats
    0.000  dogs
./data

./data/data/kaggle_dogcat/test/240.jpg
    0.999  cats
    0.001  dogs
./data/data/kaggle_dogcat/test/241.jpg
    0.828  dogs
    0.172  cats
./data/data/kaggle_dogcat/test/242.jpg
    1.000  dogs
    0.000  cats
./data/data/kaggle_dogcat/test/243.jpg
    0.999  dogs
    0.001  cats
./data/data/kaggle_dogcat/test/244.jpg
    1.000  cats
    0.000  dogs
./data/data/kaggle_dogcat/test/245.jpg
    1.000  cats
    0.000  dogs
./data/data/kaggle_dogcat/test/246.jpg
    1.000  cats
    0.000  dogs
./data/data/kaggle_dogcat/test/247.jpg
    1.000  dogs
    0.000  cats
./data/data/kaggle_dogcat/test/248.jpg
    1.000  dogs
    0.000  cats
./data/data/kaggle_dogcat/test/249.jpg
    1.000  dogs
    0.000  cats
./data/data/kaggle_dogcat/test/250.jpg
    1.000  dogs
    0.000  cats
./data/data/kaggle_dogcat/test/251.jpg
    1.000  dogs
    0.000  cats
./data/data/kaggle_dogcat/test/252.jpg
    1.000  cats
    0.000  dogs
./data/data/kaggle_dogcat/test/253.jpg
    1.000  dogs
    0.000  cats
./data

./data/data/kaggle_dogcat/test/360.jpg
    1.000  dogs
    0.000  cats
./data/data/kaggle_dogcat/test/361.jpg
    1.000  cats
    0.000  dogs
./data/data/kaggle_dogcat/test/362.jpg
    0.994  cats
    0.006  dogs
./data/data/kaggle_dogcat/test/363.jpg
    1.000  dogs
    0.000  cats
./data/data/kaggle_dogcat/test/364.jpg
    1.000  cats
    0.000  dogs
./data/data/kaggle_dogcat/test/365.jpg
    1.000  cats
    0.000  dogs
./data/data/kaggle_dogcat/test/366.jpg
    1.000  dogs
    0.000  cats
./data/data/kaggle_dogcat/test/367.jpg
    1.000  dogs
    0.000  cats
./data/data/kaggle_dogcat/test/368.jpg
    0.898  cats
    0.102  dogs
./data/data/kaggle_dogcat/test/369.jpg
    1.000  dogs
    0.000  cats
./data/data/kaggle_dogcat/test/370.jpg
    1.000  cats
    0.000  dogs
./data/data/kaggle_dogcat/test/371.jpg
    1.000  dogs
    0.000  cats
./data/data/kaggle_dogcat/test/372.jpg
    1.000  cats
    0.000  dogs
./data/data/kaggle_dogcat/test/373.jpg
    1.000  cats
    0.000  dogs
./data

In [52]:
predicts = []
ID = []
index = 0 
for f in files:
    img = image.load_img(f, target_size=(224, 224))
    if img is None:
        continue
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    pred = net.predict(x)[0]
    predicts.append(float(pred[0]))
    ID.append(index)
    index+=1

In [53]:
predicts

[1.0,
 0.000992181128822267,
 1.0,
 6.1751125046694e-13,
 1.0,
 1.0,
 2.3632985175936483e-05,
 1.0,
 0.9999994039535522,
 0.9999164342880249,
 9.222748433013184e-13,
 1.0,
 2.2568175447437392e-18,
 1.0,
 0.9999895095825195,
 0.00013963619130663574,
 7.396264590370238e-14,
 1.0,
 1.6805815903353505e-05,
 1.0,
 1.0,
 0.9999982118606567,
 1.0,
 1.0,
 1.0,
 1.6865185662507688e-16,
 3.4872016385634197e-07,
 7.246706013042115e-11,
 3.187917041458377e-09,
 2.9389504288464827e-15,
 1.4478614704729886e-13,
 3.4234828414197576e-17,
 0.9996610879898071,
 1.0,
 1.0,
 2.5474222815273333e-14,
 1.0,
 0.9999998807907104,
 5.534752656188857e-10,
 1.0,
 1.0714148701141084e-17,
 6.595630566152977e-07,
 1.0,
 1.0,
 1.0,
 7.484054265205486e-09,
 1.0,
 1.0,
 0.04730798676609993,
 1.0,
 1.0,
 1.0,
 4.011429585932649e-17,
 1.0,
 2.638354423822875e-08,
 1.8015170696695452e-18,
 1.0937363725815885e-07,
 0.9999998807907104,
 0.9999998807907104,
 1.0,
 4.62716438448074e-15,
 1.0,
 1.0,
 2.3995520237463097e-08,
 1

In [54]:
ID

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [55]:
import pandas as pd
submission = pd.DataFrame({
    'ID': ID,
    'Predicted': predicts
})
submission.to_csv("final_exam_04.csv", index=False)